In [ ]:
import sympy as sp

t = sp.Symbol('t')
q1= sp.Function("q1")(t)
q2= sp.Function("q2")(t)
# q1_dot, q2_dot, q3_dot = sp.symbols('q1_dot q2_dot q3_dot')
# q1_ddot, q2_ddot, q3_ddot = sp.symbols('q1_ddot q2_ddot q3_ddot')


# Define the link lengths and masses
m1, m2, g = sp.symbols('m1 m2 g')


# Positions of the center of mass for each link
x1 = q1
y1 = 0
x2 = x1
y2 = q2
x3 = x1 + q3
y3 = y2

# Velocities of the center of mass for each link
vx1 = sp.diff(x1, t)
vy1 = sp.diff(y1, t)
vx2 = sp.diff(x2, t)
vy2 = sp.diff(y2, t)
vx3 = sp.diff(x3, t)
vy3 = sp.diff(y3, t)


# Translational kinetic energy of each link
T1_trans = 0.5 * m1 * (vx1**2 + vy1**2)
T2_trans = 0.5 * m2 * (vx2**2 + vy2**2)
T3_trans = 0.5 * m3 * (vx3**2 + vy3**2)


# Total kinetic energy
T = T1_trans + T2_trans + T3_trans

# Potential energy of each link
V1 = 0
V2 = m2 * g * q2
V3 = m3 * g * q2

# Total potential energy
V = V1 + V2 + V3

# Lagrangian
L = T - V

# Generalized coordinates and velocities
q = [q1, q2, q3]
# q_dot = sp.Matrix([q1_dot, q2_dot, q3_dot])
# q_ddot = sp.Matrix([q1_ddot, q2_ddot, q3_ddot])
replacements = ()
for i in range(3):
    replacements += ((q[i].diff(t).diff(t), sp.Symbol(f'ddq{i + 1}')),
                    (q[i].diff(t), sp.Symbol(f'dq{i + 1}')),
                    (q[i], sp.Symbol(f'q{i + 1}')))

# print(replacements)
# Lagrangian equations
eoms = []
for i in range(3):
    L_qi = L.diff(q[i].diff(t)).diff(t) - L.diff(q[i])
    L_qi = L_qi.simplify().subs(replacements)
    eoms.append(L_qi)


# L_q2 = sp.diff(sp.diff(L, q2_dot), t) - sp.diff(L, q2)
# L_q2 = L_q2.simplify().subs({sp.diff(q2_dot, t): q2_ddot})
# L_q3 = sp.diff(sp.diff(L, q3_dot), t) - sp.diff(L, q3)
# L_q3 = L_q3.simplify().subs({sp.diff(q3_dot, t): q3_ddot})
# Pretty print the mass matrix, Coriolis matrix, and equations of motion
print("L_q1:")
sp.pretty_print(eoms[0])

print("\n L_q2")
sp.pretty_print(eoms[1])

print("\n L_q3")
sp.pretty_print(eoms[2])


q_dot = [q[0].diff(t), q[1].diff(t), q[2].diff(t)]
q_ddot = [q_dot[0].diff(t), q_dot[1].diff(t), q_dot[2].diff(t)]
# Mass matrix (inertia matrix)
M = sp.zeros(3)
M4C = sp.zeros(3)
for i in range(3):
    for j in range(3):
        M4C[i, j] = (T.diff(q_dot[i]).diff(q_dot[j])).simplify()
        M[i, j] = M4C[i, j].subs(replacements)
        # M[i, j] = sp.diff(T, q_dot[i], q_dot[j])

print("Mass Matrix (M):")
print(M)

# Coriolis matrix
C = sp.zeros(3, 3)
for i in range(3):
    for j in range(3):
        for k in range(3):
            C[i, j] += 0.5 * (M4C[i, j].diff(q[k]) + M4C[i, k].diff(q[j]) - M4C[j, k].diff(q[i])) * q_dot[k]
        C[i, j] = C[i, j].simplify().subs(replacements)


print("Coriolis Matrix (C)::")
print(C)

# Gravitational force vector
G = sp.zeros(3, 1)
for i in range(3):
    G[i] = V.diff(q[i]).simplify().subs(replacements)

print(" Gravitational Vector (G):")
print(G)
